In [0]:
try:
  %tensorflow_version 2.x # enable TF 2.x in Colab
except Exception:
  pass

In [0]:
pip install -U keras-tuner


In [0]:
# TODO: your code here
# You may add additional code cells if helpful
import numpy as np

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers

from kerastuner import RandomSearch

# Load the data
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train / 255
x_test = x_test / 255


In [0]:
# Build the model

def build_model(hp):

  num_layers = hp.Int('num_layers', 3, 6, default=3)
  lr = hp.Float('learning_rate', 1e-3, 3e-2)
  dropout_rate = hp.Float('dropout_rate', 0, 0.3, step=0.02)
  inputs = layers.Input((32, 32, 3))
  x = inputs

  for idx in range(num_layers):
      filters = hp.Int('filters_' + str(idx), 32*(idx+1), 64*(idx+1), step=32, default=64)
      x = layers.Conv2D(filters=filters, kernel_size=3, padding='same', strides=(1, 1))(x)
      x = layers.BatchNormalization()(x)
      x = layers.Activation("relu")(x)
      x = layers.Dropout(rate=dropout_rate)(x)
      x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)

  x = layers.GlobalAveragePooling2D()(x)
  outputs = layers.Dense(10, activation='softmax')(x)

  # Build model 
  model = keras.Model(inputs, outputs)
  model.compile(optimizer=Adam(lr),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model



In [0]:
!rm -rf cifar10_keras_tuner

In [0]:
########## Random search
TRIALS = 30  # number of models to train
EPOCHS = 70  # number of epoch per model
BATCH_SIZE = 64

# Initialize the tuner by passing the `build_model` function
# and specifying key search constraints: maximize val_acc (objective),
# and the number of trials to do. More efficient tuners like UltraBand() can
# be used.
tuner = RandomSearch(build_model, objective='val_accuracy', max_trials=TRIALS,
                     project_name='cifar10_keras_tuner')

# Display search space overview
tuner.search_space_summary()

In [0]:
from kerastuner.tuners import Hyperband

########## Hyperband search
TRIALS = 2  
EPOCHS = 70  # number of epoch per model
BATCH_SIZE = 64

# Initialize the tuner by passing the `build_model` function
# and specifying key search constraints: maximize val_acc (objective),
# and the number of trials to do. More efficient tuners like UltraBand() can
# be used.
tuner = Hyperband(build_model, objective='val_accuracy', hyperband_iterations=TRIALS,
                  max_epochs=EPOCHS, project_name='cifar10_tuner_hyperband')

# Display search space overview
tuner.search_space_summary()

In [0]:
# Perform the model search. The search function has the same signature
# as `model.fit()`.
early_stopping_cb = keras.callbacks.EarlyStopping(patience=40,
                                                  restore_best_weights=True)     
tuner.search(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS,
             validation_data=(x_test, y_test), 
                    callbacks=[early_stopping_cb])

In [0]:
### Hyperband search
# Display the best models, their hyperparameters, and the resulting metrics.
tuner.results_summary()

# Retrieve the best model and display its architecture
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

In [0]:
### Random search
# Display the best models, their hyperparameters, and the resulting metrics.
tuner.results_summary()

# Retrieve the best model and display its architecture
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()